In [0]:
!git clone https://github.com/vineetdhanawat/twitter-sentiment-analysis.git

fatal: destination path 'twitter-sentiment-analysis' already exists and is not an empty directory.


In [0]:
!pip3 install tensorflow-hub #babelnet

In [0]:
import tensorflow_hub as hub
import tensorflow as tf
from keras import backend as K

sess = tf.Session()
K.set_session(sess)
# 세션 초기화. 이는 텐서플로우 개념.

elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)
# 텐서플로우 허브로부터 ELMo를 다운로드

sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [0]:
import pandas as pd
data = pd.read_csv('twitter-sentiment-analysis/datasets/Sentiment Analysis Dataset.csv', encoding='latin-1')

data = data[:10000]

# Sentiment 레이블(0(긍정), 1(부정)로 구성)
# SentimentText 입력문장

#data['Sentiment'] = data['Sentiment'].replace(['good','bad'],[0,1])
y_data = list(data['Sentiment'])
X_data = list(data['SentimentText'])

In [0]:
print(len(y_data))

10000


In [0]:
# 학습, 테스트 데이터 80%, 20%로 구분.
print(len(X_data))
n_of_train = int(len(X_data) * 0.8)
n_of_test = int(len(X_data) - n_of_train)
print(n_of_train)          
print(n_of_test)
type(X_data)

10000
8000
2000


list

In [0]:
import numpy as np
X_train = np.asarray(X_data[:n_of_train]) 
y_train = np.asarray(y_data[:n_of_train]) 
X_test = np.asarray(X_data[n_of_train:]) 
y_test = np.asarray(y_data[n_of_train:])

In [0]:
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), as_dict=True, signature="default")["default"]
# 데이터의 이동이 케라스 → 텐서플로우 → 케라스가 되도록 하는 함수

In [0]:
# 
from keras.models import Model
from keras.layers import Dense, Lambda, Input

# keras ELMO + dense, sigmoid, binary classification 모델 선언
input_text = Input(shape=(1,), dtype=tf.string)
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
hidden_layer = Dense(256, activation='relu')(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(hidden_layer)     
model = Model(inputs=[input_text], outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  ## 0에가깝냐 1에가깝냐 하는거,     adam 케라스에서 물고 있는거

W0802 06:37:30.629973 139891906656128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0802 06:37:30.632789 139891906656128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0802 06:37:31.044530 139891906656128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0802 06:37:31.075361 139891906656128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0802 06:37:31.098263 139891906656128 deprecation_wrappe

In [0]:
# 학습 진행
history = model.fit(X_train, y_train, epochs=1, batch_size=60)

W0802 06:37:50.560878 139891906656128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/1
8000/8000 [==============================] - 1061s 133ms/step - loss: 0.5262 - acc: 0.7371


In [0]:
# 모델 성능 비교.
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))

2000/2000 [==============================] - 365s 182ms/step

 테스트 정확도: 0.7255


In [0]:
## 73%정도나옴된다